# pytRIBS

## The Need For pytRIBS
pytRIBS was designed to aid users of the TIN-based Real-time Integrated Basin Simulator ([tRIBS](https://tribshms.readthedocs.io/en/latest/)) distributed hydrologic model in model setup, execution, and result analysis. Prior to pytRIBS, setting up a tRIBS model could take multiple days, involved a number of proprietary software programs, and was prone to user error. pytRIBS addresses all of these challenges, letting users approach hydrologic modeling in a programmatic and efficient manner.

![tRIBS Workflow](../smf_assets/tRIBS_workflow_horz.png)

**Figure 1.** A tRIBS workflow consists of three main steps: (1) Collate and generate model inputs. (2) Run the model simulation(s). (3) Review and analyze results. The first and last step (denoted by red text and dashed lines) are user intensive activities that challenge reproducibility and are often very time consuming. The asterisks denote steps required for the parallel operation of the model. 

## pytRIBS Design

!['pytRIBS Design'](../smf_assets/pytRIBS_design.png)

**Figure 2.** pytRIBS uses an object-oriented approach to capture the major components and steps required for setting up, simulating, and analyzing a tRIBS model. The preprocessing classes are intended to expedite the first step in setting up a tRIBS model, whereas the simulation classes provide users with tools to effectively run and analyze the model through a Python interface. The Project class is not directly linked to another class as it is limited to storing directory information and meta data. Only select attributes or methods are shown for each class, for a full list of attributes and methods see the associated documentation. 
*Water balance can be calculated for both the basin averaged condition or at individual nodes. 
**Mesh class relies on Preprocess and MeshGenerator classes accessed via these instances.

# Newman Canyon Full Example
The following section documents how pytRIBS can efficiently generate a locally refined TIN mesh for the TIN-based real-time integrated basin simulator (tRIBS). Here we use the Newman Canyon watershed located near Flagstaff, Arizona, USA as an example. Newman Canyon is ~60 km$^2$ with an average elevation of 2230 m.

## Imports

In [ ]:
# note you can install pytRIBS via pip; see: https://pypi.org/project/pytRIBS/
from pytRIBS.classes import *

In [ ]:
# if you have installed pytRIBS, the following libraries should already be in your environment
import os, sys, shutil
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from shapely.ops import unary_union
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors

## Project Class: Setup Model Information

In [ ]:
name ='Newman_Canyon'
epsg = 26912
proj = Project(os.getcwd(),name,epsg) # Create an instance of a Project Class

In [ ]:
proj.directories

In [ ]:
proj.meta

The code below code copies in the needed data sets to complete this tutorial.

In [ ]:
dem = '../newman_canyon_init_data/DEM_USGS3m.tif'
shutil.copy(dem, proj.directories['preprocessing'])
dem = f"{proj.directories['preprocessing']}/{os.path.basename(dem)}"

canopy_height_data = '../newman_canyon_init_data/meta_treeheights_3m.asc'
shutil.copy(canopy_height_data, proj.directories['land'])
canopy_height_data = f"{proj.directories['land']}/{os.path.basename(canopy_height_data)}"

## Mesh Class: Process DEM and Generate Mesh

### Preprocessing
The Mesh class can be initiated with no arguments, but here we demonstrate how to initiate the class with arguments for the Preprocessing and MeshGeneration component classes. 

In [ ]:
# Preprocessing Arguments
verbose_mode=False# suppresses whitebox output

# UTM coordinates for outlet/pour point
x = 455332
y = 3879538

snap_dis = 100 # allowable distance in m for snapping outlet points to stream network
threshold_area = 1e6 # area in m^2 for determining stream network

# tuple of preprocessing arguments to be passed into the the mesh class
preprocess_args = ([x,y],snap_dis, threshold_area, dem, verbose_mode,proj.meta,proj.directories['preprocessing'])

In [ ]:
# Mesh Generation Arguments
# output directory can be specified in Preproceesing, but if not provided preprocessing is the default directory 
output_dir = proj.directories['preprocessing'] 

path_to_raster = f'{output_dir}/{name}_clipped_ext.tif' # these are default outputs but can be further modified. See documentation.
path_to_watershed = f'{output_dir}/{name}_boundary.shp'
path_to_stream_network = f'{output_dir}/{name}_stream.shp'
path_to_outlet = f'{output_dir}/{name}_outlet.shp'
maxlevel= 8 # This can be set to none, if so the maximum level possible wiil be used. 

mesh_generation_args = (path_to_raster, path_to_watershed, path_to_stream_network, path_to_outlet, maxlevel)

In [ ]:
tmesh = Mesh(preprocess_args=preprocess_args, generate_mesh_args=mesh_generation_args,meta=proj.meta)

Now, we have created all the files necessary for generating a mesh. Below we provide a visualization of these data. Note you can read these in independently, but they are all ready assigned to ```tmesh.mesh_generator``` or alternatively can be used from the MeshGeneration class.

#### Example Figure 1: Preprocessing products produced by pytRIBS

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

extent = tmesh.mesh_generator.get_extent()

# Display the image
img = ax.imshow(
    tmesh.mesh_generator.data,
    extent=extent,
    cmap='terrain'
)

# Add colorbar
cbar = fig.colorbar(img, ax=ax, orientation='vertical')
cbar.set_label('Data Value', fontsize=14)
cbar.ax.tick_params(labelsize=12)

# Plot watershed boundary
watershed_handle = tmesh.mesh_generator.watershed.plot(
    ax=ax,
    facecolor='none',
    edgecolor='red',
    linewidth=1.5,
    label='Watershed Boundary'
)

# Plot stream network
stream_network_handle = tmesh.mesh_generator.stream_network.plot(
    ax=ax,
    facecolor='none',
    edgecolor='black',
    linewidth=1,
    label='Stream Network'
)

# Plot outlet
outlet_handle = tmesh.mesh_generator.outlet.plot(
    ax=ax,
    facecolor='none',
    edgecolor='yellow',
    marker='*',
    markersize=100,
    label='Outlet'
)

# Set axis labels and title
ax.set_xlabel('Easting (UTM)', fontsize=14)
ax.set_ylabel('Northing (UTM)', fontsize=14)
ax.set_title('Watershed, Stream Network, and Outlet', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=12)

# Create legend
legend_elements = [
    Line2D([0], [0], color='red', lw=1.5, label='Watershed Boundary'),
    Line2D([0], [0], color='black', lw=1, label='Stream Network'),
    Line2D([0], [0], color='yellow', marker='*', markersize=10, linestyle='None', label='Outlet')
]

ax.legend(handles=legend_elements, loc='upper left', fontsize=12)

# Adjust layout and save figure
plt.tight_layout()

### Mesh Generation
With this data we can generate a locally refined mesh rapidly and easily using a Harr wavelet transform. Below we create the points file required for a tRIBS model simulation and visualize the associated mesh using PyVista.

In [ ]:
threshold = 0.5

In [ ]:
#note we'll use buffered watershed for other workflows to ensure data exists outside of the mesh
points, buffered_watershed = tmesh.mesh_generator.extract_points_from_significant_details(threshold)

In [ ]:
buffered_watershed = buffered_watershed.buffer(250*2)

In [ ]:
# save out the buffered watershed in case you need it later
gdf = gpd.GeoDataFrame({'id':[1]}, geometry=[buffered_watershed], crs=tmesh.meta['EPSG'])
gdf.to_file(f"{proj.directories['preprocessing']}/{name}_buffered_watershed.shp")

In [ ]:
# write out points to point file
tmesh.pointfilename['value'] = proj.directories['mesh']+'/'+name+'.points'
gdf = tmesh.mesh_generator.convert_points_to_gdf(points)
tmesh.mesh_generator.write_point_file(gdf,tmesh.pointfilename['value'])

### Mesh Partition

Next we need to partition the mesh so we can run tRIBS in parallel. To do this we will run a docker image of meshbuilder in the folder ```/data/mesh```. But first we need an associated input file for MeshBuilder, which only requires the key word POINTFILENAME and is accomplished using the ```generate_meshbuild_input_file``` method from MeshGeneration.

In [ ]:
mesh_par_in = 'meshbuild.in'
par_dir = f"{proj.directories['mesh']}/{mesh_par_in}"
tmesh.mesh_generator.generate_meshbuild_input_file(par_dir,name,point_filename=f'{name}.points')

In [ ]:
partition_args = [mesh_par_in,4,1,name]

In [ ]:
tmesh.mesh_generator.partition_mesh(f"{proj.directories['mesh']}/", partition_args)

In [ ]:
tmesh.graphfile['value'] = 'data/model/mesh/Newman_Canyon_flow_4nodes.reach'

In [ ]:
tmesh.graphoption['value'] = 1

## Soil Class: Obtain and Generate tRIBS Soil Parameters

The setup for this example is fairly simple. All that is required is to provide the shape file for the watershed boundary and the EPSG code for the project (which can be provided from the Mesh class in the Newman Canyon Mesh Example). There are some items in the soil workflow however that require expert knowledge, in particular, it's expected that the user provides X,Y,Z values for the range of soil classes produced for the work flow.

In [ ]:
soil = Soil(meta=proj.meta)

In [ ]:
soil.gwaterfile['value'] = f"{proj.directories['soil']}/{name}_watertable.iwt"
soil.generate_uniform_groundwater(buffered_watershed,2900)

The following line of code constitutes the bulk of the preprocessing analysis conducted by the Soil class. This workflow:
1) Downloads Soil Grids 250 data and fills in nan values
2) Generate the following tRIBS parameters 'Ks', 'theta_r', 'theta_s',' psib', 'm' and 'f' grids (see [here](https://tribshms.readthedocs.io/en/latest/index.html#) for more details).
3) Creates a soil class map (\*.soi) and soil description table (\*.sdt) and grid file (\*.gdf).

Note the underlying methods accessed by `run_soil_workflow` can alternatively be accessed for additional fine-tuning or control over the process. As of now the Soil Grids 250 data is directly downloaded into a sub-directory _sg250_.

In [ ]:
soil.run_soil_workflow(buffered_watershed,proj.directories['soil'])

Here we read in the soil table produced by the above workflow and update specific parameters for the provided soil class and texture. Below we first read in the created table and map, visualize the map, and then assign saturated anistropy ration (As), unsaturated ansitropy ration (Au), volumetric heat conductivity (ks), and soil heat capacity (Cs) values to the classes. Gridded parameters produced by the run_soil_workflow() in the soil table have a corresponding no data value of 9999.99.

In [ ]:
# only need to run if the workflow was skipped
# soil.soilmapname['value'] = f"{proj.directories['soil']}/sg250/soil_classes.soi"
# soil.scgrid['value'] = f"{proj.directories['soil']}/scgrid.gdf"
# soil.soiltablename['value'] = f"{proj.directories['soil']}/soils.sdt"

In [ ]:
soil_table = soil.read_soil_table(textures=True)
soil_map = InOut.read_ascii(soil.soilmapname['value'])

After we have read in the soil table, we can see the associated class ID and texture as follows.

In [ ]:
for cls in soil_table:
    print(f"Class ID and texture: {cls['ID']}, {cls['Texture']}")

In [ ]:
for cls in soil_table:
    cls['As'] = 300
    cls['Au'] = 900
    cls['PsiB'] = -30 # temporary... adjustment, need to debug
    cls['ks'] = 0.7 #J/msK
    cls['Cs'] = 1.4e6 # J/m^3k

In [ ]:
soil.write_soil_table(soil_table,soil.soiltablename['value'],textures=True)

In [ ]:
soil.optsoiltype['value'] = 1

#### Example Figure 3: Example soil classification map generated from pytRIBS Soil Class

In [ ]:
transform = soil_map['profile']['transform']

x_min = transform[2]
x_max = x_min +  soil_map['profile']['width'] * transform[0]
y_max = transform[5]
y_min = y_max + soil_map['profile']['height']* transform[4]  # Pixel height is negative

extent = [x_min, x_max, y_min, y_max]

fig, ax = plt.subplots(figsize=(10, 8))

img = ax.imshow(
    soil_map['data'],
    extent=extent,
    cmap=soil.discrete_colormap(4,'cividis')
)
cbh = plt.colorbar(img, ax=ax, ticks=range(int(soil_map['data'].min()), int(soil_map['data'].max()) + 1))
cbh.set_label('Soil Class', fontsize=14)
cbh.ax.yaxis.set_major_locator(mticker.MultipleLocator(1))  # Ensure discrete ticks
tmesh.mesh_generator.watershed.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=1.5)
ax.set_xlabel('Easting (UTM)', fontsize=14)
ax.set_ylabel('Northing (UTM)', fontsize=14)
ax.set_xlim([tmesh.mesh_generator.watershed.bounds.minx[0], tmesh.mesh_generator.watershed.bounds.maxx[0]])
ax.set_ylim([tmesh.mesh_generator.watershed.bounds.miny[0], tmesh.mesh_generator.watershed.bounds.maxy[0]])
ax.tick_params(axis='both', which='major', labelsize=12)
ax.set_title('Soil Classification Map', fontsize=16)
plt.tight_layout()


## Met Class: Obtain and Process NLDAS-2 Meterological Forcing Data
Meteorological forcing for the tRIBS model can be supplied either as point-based station data or as raster data. Managing these data manually can be time-consuming and susceptible to errors. To simplify this process, the Met class allows users to efficiently obtain meteorological forcing data from the North American Land Data Assimilation System for a specified location and time period. 

In [ ]:
met = Met(meta=proj.meta)
met.hydrometbasename['value'] = name
met.hydrometstations['value'] = f"{proj.directories['met_meteor']}/{name}.sdf"
met.gaugestations['value'] =f"{proj.directories['met_precip']}/{name}.sdf"

In [ ]:
end = '2021-10-01'
begin = '2019-09-30'

In [ ]:
mean_elevation = gdf.elevation.mean()

In [ ]:
# As of Nov 2025 the met workflow is not functional due to change in the pynldas2 dependencies
#met_df = met.run_met_workflow(buffered_watershed,begin,end,mean_elevation)

# So lets just copy met data that was previously generated for now
met_data = '../newman_canyon_init_data/met/meteor/met_Newman_Canyon_1.mdf'
shutil.copy(met_data, proj.directories['met_meteor'])
met_data = '../newman_canyon_init_data/met/meteor/Newman_Canyon.sdf'
shutil.copy(met_data, proj.directories['met_meteor'])

precip_mdf = '../newman_canyon_init_data/met/precip/precip_Newman_Canyon_1.mdf'
shutil.copy(precip_mdf, proj.directories['met_precip'])
precip_mdf = '../newman_canyon_init_data/met/precip/Newman_Canyon.sdf'
shutil.copy(precip_mdf, proj.directories['met_precip'])

In [ ]:
msdf = met.read_met_sdf(met.hydrometstations['value'])

In [ ]:
mmdf = met.read_met_station(msdf[0]['file_path'])

#### Example Figure 5: NLDAS wind speed conversion

In [ ]:
# This plotting code is not functional until the met workflow has been fixed
""" fig, ax = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

ax[0].plot(met_df.index, met_df.wind_u, color='blue', linestyle='-', linewidth=1.5)
ax[0].set_ylabel('10 m Zonal windspeed (m/s)')
ax[0].grid(True)
ax[0].legend(['Wind U'], loc='upper right')

ax[1].plot(met_df.index, met_df.wind_v, color='red', linestyle='-', linewidth=1.5)
ax[1].set_ylabel('10 m Meridional windspeed (m/s)')
ax[1].grid(True)
ax[1].legend(['Wind V'], loc='upper right')

ax[2].plot(mmdf.date, mmdf.US, color='green', linestyle='-', linewidth=1.5)
ax[2].set_ylabel('2 m wind speed (m/s)')
ax[2].set_xlabel('Date')
ax[2].grid(True)
ax[2].legend(['tRIBS Data'], loc='upper right')

ax[2].xaxis.set_major_locator(mdates.MonthLocator(bymonthday=[1]))
ax[2].xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

plt.setp(ax[2].xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout() """

## Land Class: Create Land Cover Map and Assign tRIBS Land Cover Parameters
tRIBS supports spatially varying land cover parameters, which can be provided through a classification map and table, raster data, or a combination of both. These inputs may remain static over time or vary across different periods (e.g., monthly, seasonally, annually). The Land class offers various attributes and methods to manage these parameters, although the availability and resolution of land cover data often differ significantly between applications, necessitating more user input. Consequently, the Land class does not offer an all-encompassing workflow but instead provides a set of tools to manage data from various sources and helper functions to generate the necessary input files for a tRIBS simulation. For example, the classify_vegetation_height() method can be applied to lidar-derived products, such as the 1-m resolution canopy height data from Tolan et al. (2024). Analysis of this dataset reveals a bimodal distribution with a low point at 4.5 meters. Using this threshold, the data can be divided into two groups: values below 4.5 meters are classified as grass and/or shrub, while those above are classified as ponderosa pine. This pine class can then be further subdivided using parameters like leaf area index, which vary with height. While this method demonstrates how classification can be performed, expert judgment should be applied to ensure proper use of thresholds.

In [ ]:
land = Land(meta=proj.meta)

In [ ]:
veg_height_data = InOut.read_ascii(canopy_height_data)
veg_height = veg_height_data['data'].flatten()

In [ ]:
np.unique(veg_height)

In [ ]:
threshold = [(0,0,1),(0,np.exp(1.5),2),(np.exp(1.5),np.exp(2.25),3),(np.exp(2.25),np.max(veg_height),4)]

In [ ]:
land.landmapname['value'] = f"{proj.directories['land']}/{name}_landcover.asc"
classes, ldt  = land.classify_vegetation_height(canopy_height_data,threshold,land.landmapname['value'],plot_result=False)

In [ ]:
# values are loosely sourced from Cederstrom et al. 2024, here we increment certain parameters with higher tree height
for cnt,n in enumerate(range(1,4)):
    ldt[n]['a'] = 0.3  # dummy placeholders not used w/ selected model options
    ldt[n]['b1'] = 0.3 # dummy placeholders not used w/ selected model options
    ldt[n]['h'] = 0.3 # dummy placeholders not used w/ selected model options
    ldt[n]['b1'] = 0.3
    ldt[n]['P'] = 0.3
    ldt[n]['S'] = 2.5
    ldt[n]['K'] = .12
    ldt[n]['b2'] = 3.0
    ldt[n]['Kt'] = 0.4
    ldt[n]['Al'] = 0.3 - cnt/10
    ldt[n]['Rs'] = 10 + cnt*5
    ldt[n]['V'] = 0.80 + (50+cnt*5)/100
    ldt[n]['LAI'] = 2.0 + cnt*0.75
    ldt[n]['theta*_s'] = 0.38
    ldt[n]['theta*_t'] = 0.38

In [ ]:
# update grasses informed from Mahmood and Vivoni 2014
ldt[0]['a'] = 0.3  # dummy placeholders not used w/ selected model options
ldt[0]['b1'] = 0.3 # dummy placeholders not used w/ selected model options
ldt[0]['h'] = 0.3 # dummy placeholders not used w/ selected model options
ldt[0]['P'] = 0.1
ldt[0]['S'] = 1
ldt[0]['K'] = .12
ldt[0]['b2'] = 4.7
ldt[0]['Al'] = 0.3
ldt[0]['Kt'] = 0.9
ldt[0]['Rs'] = 40
ldt[0]['V'] = 0.8
ldt[0]['LAI'] = 1
ldt[0]['theta*_s'] = 0.38
ldt[0]['theta*_t'] = 0.38

In [ ]:
land.landtablename['value'] = f"{proj.directories['land']}/{name}.ldt"
land.lugrid['value'] = f"{proj.directories['land']}/{name}.gdf"

In [ ]:
land.write_landuse_table(ldt,land.landtablename['value'] )

In [ ]:
parameters = [{'Variable Name':'VH', 'Raster Path':canopy_height_data[:canopy_height_data.find('.')],'Raster Extension':'asc'}]
land_gdf_content = land.create_gdf_content(parameters,buffered_watershed)

In [ ]:
land.write_grid_data_file(land.lugrid['value'],land_gdf_content)

In [ ]:
land.update_landfiles_with_dates(canopy_height_data, begin)

#### Example Figure 6: Vegetation Height Distribution and Landcover Classification Map

In [ ]:
plt.hist(np.log(veg_height[veg_height>0]))
plt.xlabel('Log(Vegetation Height)')
plt.ylabel('Frequency')

In [ ]:
transform = veg_height_data['profile']['transform']

x_min = transform[2]
x_max = x_min +  veg_height_data['profile']['width'] * transform[0]
y_max = transform[5]
y_min = y_max + veg_height_data['profile']['height']* transform[4]  # Pixel height is negative

extent = [x_min, x_max, y_min, y_max]


fig, ax = plt.subplots(figsize=(10, 8))

img = ax.imshow(
    classes,
    extent=extent,
    cmap=land.discrete_colormap(4,'viridis')
)

cbh = plt.colorbar(img, ax=ax, ticks=range(int( classes.min()), int( classes.max()) + 1))
cbh.set_label('Land Class', fontsize=14)
cbh.ax.yaxis.set_major_locator(mticker.MultipleLocator(1))  # Ensure discrete ticks
tmesh.mesh_generator.watershed.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=1.5)
ax.set_xlabel('Easting (UTM)', fontsize=14)
ax.set_ylabel('Northing (UTM)', fontsize=14)
ax.set_xlim([tmesh.mesh_generator.watershed.bounds.minx[0], tmesh.mesh_generator.watershed.bounds.maxx[0]])
ax.set_ylim([tmesh.mesh_generator.watershed.bounds.miny[0], tmesh.mesh_generator.watershed.bounds.maxy[0]])
ax.tick_params(axis='both', which='major', labelsize=12)
ax.set_title('Land Classification Map', fontsize=16)
plt.tight_layout()

## Model Class: Pre-flight Check  and Model Simulation
The Model Class allows users to modify tRIBS model inputs, validate that all inputs are appropriate for the chosen options, and run the tRIBS model directly using Docker and the Docker SDK for Python. It can be initialized with or without preprocessing classes, providing a flexible approach to numerical experiments, and also supports manual setup or starting through an existing input file.

In [ ]:
model = Model(met=met,land=land,soil=soil,mesh=tmesh,meta=proj.meta)

In [ ]:
# mesh
model.parallelmode['value'] = 1 # set to run in parallel ## update graph file too!!

# soil 
model.depthtobedrock['value'] = 3 # constant bedrock depth set to 3 meters

#land
model.optlanduse['value'] = 1 # needed for reading in canopy height

#simulation variables
model.startdate['value'] = '09/30/2019/00/00'
model.runtime['value'] = 8760*2 # two years
model.outfilename['value'] = f"{proj.directories['results']}/{name}"
model.outhydrofilename['value'] = f"{proj.directories['results']}/{name}"

In [ ]:
# create node list file for visualizing individual node output
node_ids = [1960, 1547, 3682]
model.write_node_file(node_ids,'data/model/nodes.dat')
model.nodeoutputlist['value'] = 'data/model/nodes.dat'

In [ ]:
model.check_paths()

In [ ]:
input_file = f'{name}.in'

In [ ]:
model.write_input_file(input_file)

In [ ]:
model.run_tribs_docker(os.getcwd(),input_file,execution_mode='parallel',num_processes=4)

## Results Class: Merge and Visualize Results
The Results class simplifies working with tRIBS outputs by offering post-processing methods that handle everything from file management to basic model output analysis. tRIBS generates a large amount of data with fine spatial and temporal resolutions, including time series of streamflow and spatially averaged state and flux variables. Additionally, the model produces Voronoi diagrams that can be used with both dynamic snapshots (captured at specific times) and integrated outputs (aggregated over the entire model run). The Results class helps manage these outputs, providing users with tools to merge parallel results and perform further analysis using commonly utilized data libraries.

In [ ]:
results = Results('Newman_Canyon.in',meta=proj.meta)

In [ ]:
results.options['templapse']

In [ ]:
gdf = results.voronoi.merge(results.int_spatial_vars,on='ID')
results.get_mrf_results()
results.get_element_results()
results.get_mrf_water_balance('cold_warm')

mrf = results.mrf['mrf']
mrf.set_index('Time',inplace=True)
wb = results.mrf['waterbalance']

In [ ]:
gdf['ET'] = gdf.cET/8760*2

In [ ]:
gdf['persTime'] = gdf['cHrsSnow']/(8760*2)

#### Example Figure 7: Example application of Results Class--peak SWE and time series 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

highlight_ids = node_ids 

gdf.plot(column='peakWE', cmap='BuPu', legend=True, ax=ax[0])

highlighted_gdf = gdf[gdf['ID'].isin(highlight_ids)] 
highlighted_gdf.plot(ax=ax[0], edgecolor='red', facecolor='none', linewidth=2, label='Highlighted Polygons')

merged_outline = unary_union(gdf.geometry) 
gpd.GeoSeries([merged_outline]).plot(ax=ax[0], edgecolor='black', facecolor='none', linewidth=2, label='Merged Outline')

ax[0].set_title('Peak Water Equivalent (WE)', fontsize=14)
ax[0].set_xlabel('Longitude', fontsize=12)
ax[0].set_ylabel('Latitude', fontsize=12)

for node in node_ids:
    elem = results.element[node]['pixel']
    ax[1].plot(elem['Time'], elem['SnWE_cm'], label=f"Node {node}") 

ax[1].set_xlabel("Time", fontsize=14) 
ax[1].set_ylabel("SnWE (cm)", fontsize=14)  
ax[1].set_title("SnWE over Time for Each Node", fontsize=16) 
ax[1].tick_params(axis='x', rotation=45)  
ax[1].legend(loc='best', fontsize=12)  
ax[1].grid(True, linestyle='--', alpha=0.5)

fig.tight_layout()

#### Example Figure 8: Example application of Results Class: ET and Topographic Wetness Index

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

gdf.plot(ax=ax[0], column='ET', cmap='YlOrBr',  legend=True)

ax[0].set_title('Evapotranspiration (ET) Map', fontsize=14)
ax[0].set_xlabel('Longitude', fontsize=12)
ax[0].set_ylabel('Latitude', fontsize=12)

gpd.GeoSeries([merged_outline]).plot(ax=ax[0], edgecolor='black', facecolor='none', linewidth=2, label='Merged Outline')

sc = ax[1].scatter(np.log((gdf['CAr'] * 1e6 / gdf['FWidth']) / gdf['Slp']),
                   gdf['ET'], c=gdf['ET'], cmap='YlOrBr', alpha=0.75, edgecolor='k')

ax[1].set_title('ET vs Topographic Wetness Index', fontsize=14)
ax[1].set_xlabel('TWI', fontsize=12)
ax[1].set_ylabel('Evapotranspiration (ET)', fontsize=12)
ax[1].grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()

#### Example Figure 9: Visualization of sub-watershed partitioning for parallel operations

In [ ]:
ax = gdf.plot(column='processor',legend=True,cmap=results.discrete_colormap(4,'Set2'))
gpd.GeoSeries([merged_outline]).plot(ax=ax, edgecolor='black', facecolor='none', linewidth=2, label='Merged Outline')